In [ ]:
 ! pip install -q kaggle

In [ ]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sruthikuriak","key":"99434af76be232f0a50bba71aa5d475c"}'}

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
 ! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d andrewmvd/face-mask-detection 

 99% 394M/398M [00:13<00:00, 35.5MB/s]
100% 398M/398M [00:13<00:00, 30.5MB/s]


In [ ]:
! pip install --upgrade tensorflow
! pip install --upgrade tensorflow-addons

In [ ]:

# to deal with file system
import os
# for reading images
import cv2
# to read and process xml files
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

# preprocessing images
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

# for train and validation split
from sklearn.model_selection import train_test_split

# modelling with VGG19
from tensorflow.keras.applications import  VGG19
model_name = 'vgg_19'
from tensorflow.keras.applications.vgg19  import preprocess_input as vgg_preprocess_input
from tensorflow.keras import Model
from tensorflow.keras.layers import MaxPooling2D, AveragePooling2D, Dropout, BatchNormalization, Flatten, Dense, Input
from tensorflow.keras.optimizers import Adam


# for callbacks
import time
from tensorflow.keras.callbacks import Callback, ModelCheckpoint, EarlyStopping
from datetime import datetime


# model evaluation
from tensorflow.keras.metrics import Recall, Precision
from tensorflow_addons.metrics import F1Score
from sklearn.metrics import classification_report, confusion_matrix


In [ ]:
! unzip /content/face-mask-detection.zip

In [ ]:
# setting some parameters
image_size = 224 # this is the size that gave me better results than default size of 224
validation_split_size = 0.20 # 20% will be used for validation

# define the hyperparamets for training the neural network
batch_size = 32
init_lr = 0.0_001
num_epochs = 100

# directories
labels_path = '/content/annotations/'
images_path = '/content/images/'

#!mkdir './VGG19'
save_vgg19 = './VGG19/'

mkdir: cannot create directory ‘./VGG19’: File exists


In [ ]:
images = sorted(os.listdir("../content/images/"))
labels = sorted(os.listdir("../content/annotations/"))


len(images) == len(labels), len(images), len(labels)

(True, 853, 853)

In [ ]:
def generate_label_dictionary(xml_loc): 
    """
    takes location to image and xml files on file system and return the image as numpy array and extracted bounding boxes
    """
    with open(xml_loc) as xml_file:
        # read the input file
        soup = BeautifulSoup(xml_file.read(), 'xml')
        objects = soup.find_all('object')

        # extract the number of persons in an image
        num_persons = len(objects)

        # to store all the points for boundary boxes and target labels
        boxes = []
        labels = []
        # doing it now
        for obj in objects:
            # extract output class and append it to 'boxes' list
            if obj.find('name').text == "without_mask":
                labels.append(0)
            elif obj.find('name').text == "mask_weared_incorrect":
                labels.append(1)
            elif obj.find('name').text == "with_mask":
                labels.append(2)
            else:
                break
            
            # extract bounding box and append it to 'labels' list
            xmin = int(obj.find('xmin').text)
            ymin = int(obj.find('ymin').text)
            xmax = int(obj.find('xmax').text)
            ymax = int(obj.find('ymax').text)
            boxes.append([xmin, ymin, xmax, ymax])
        

        # converting them to numpy arrays
        boxes = np.array(boxes)
        labels = np.array(labels)

        # save them to dictionary
        target = {}
        target["labels"] = labels
        target["boxes"] = boxes

        return target, num_persons

In [ ]:
targets=[] # store coordinates of bounding boxes
num_persons=[] # stores number of faces in each image

#run the loop for number of images we have
for label_path in labels:
    # generate label
    target_image, num_persons_image = generate_label_dictionary(labels_path+label_path)
    targets.append(target_image)
    num_persons.append(num_persons_image)

In [ ]:
print(len(targets))
print()
print(targets[0: 100: 10])
print()
print(num_persons[0: 100: 10])

853

[{'labels': array([0, 2, 0]), 'boxes': array([[ 79, 105, 109, 142],
       [185, 100, 226, 144],
       [325,  90, 360, 141]])}, {'labels': array([2]), 'boxes': array([[121, 192, 212, 312]])}, {'labels': array([2, 2, 2, 2, 2]), 'boxes': array([[116,  88, 150, 122],
       [160,  79, 193, 118],
       [235,  43, 272,  87],
       [304,  68, 336, 102],
       [379,  61, 399,  96]])}, {'labels': array([0]), 'boxes': array([[118, 151, 216, 279]])}, {'labels': array([2, 0, 2, 2, 2, 0, 2]), 'boxes': array([[ 92,  19, 117,  50],
       [156,  22, 177,  48],
       [179,  24, 197,  47],
       [183,  74, 205, 100],
       [269,  70, 285,  89],
       [343,  15, 359,  34],
       [302,  43, 318,  59]])}, {'labels': array([2]), 'boxes': array([[102, 200, 194, 333]])}, {'labels': array([2, 2, 2, 2, 2, 2, 2]), 'boxes': array([[ 66,  25,  82,  48],
       [101,  41, 124,  68],
       [173,  34, 193,  60],
       [204,  75, 225,  99],
       [289,  25, 312,  54],
       [350,  17, 368,  39],
  

In [ ]:
face_images = []
face_labels = []

# read each image from the file system and extract only the faces using the boundaries extracted in previous step
for i, image_path in enumerate(images):
    image_read = cv2.imread(images_path+image_path, cv2.IMREAD_COLOR)
    # get co-ordinates of the image
    for j in range(0, num_persons[i]):
        # get the locations of boundary box now
        face_locs = targets[i]['boxes'][j]
        # extract the face now using those co-ordinates
        temp_face = image_read[face_locs[1]:face_locs[3], face_locs[0]:face_locs[2]]
        temp_face = cv2.resize(temp_face, (image_size, image_size))
        temp_face = vgg_preprocess_input(temp_face)
        
        # store this processed image to list now
        face_images.append(temp_face)
        # store it's respective label too
        face_labels.append(targets[i]['labels'][j])

# convert them to numpy arrays
face_images = np.array(face_images, dtype=np.float32)
face_labels = np.array(face_labels)
print(face_images.shape, face_labels.shape)

(4072, 224, 224, 3) (4072,)


In [ ]:
np.unique(face_labels, return_counts=True)

(array([0, 1, 2]), array([ 717,  123, 3232]))

In [ ]:
def show_face_and_label(index):
    plt.imshow(face_images[index])
    plt.show()

    face_label_num = face_labels[index]

    if face_label_num == 0:
        face_label_text = "doesn't have a mask on."
    elif face_label_num == 1:
        face_label_text = "wore mask improperly."
    elif face_label_num == 2:
        face_label_text = "has a mask on."
    else:
        face_label_text = "error"
    return 'person {}'.format(face_label_text)

In [ ]:
#show_face_and_label(2)
#show_face_and_label(46)
#show_face_and_label(47)

In [ ]:
# since one-hot encoding need to be done for 
face_labels_enc = to_categorical(face_labels)
face_labels_enc

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

In [ ]:
pd.DataFrame(face_labels_enc).apply(pd.Series.value_counts, normalize=False).to_dict()

{0: {0.0: 3355, 1.0: 717}, 1: {0.0: 3949, 1.0: 123}, 2: {0.0: 840, 1.0: 3232}}

In [ ]:
pd.DataFrame(face_labels_enc).apply(pd.Series.value_counts, normalize=True).to_dict()

{0: {0.0: 0.8239194499017681, 1.0: 0.17608055009823181},
 1: {0.0: 0.9697937131630648, 1.0: 0.030206286836935166},
 2: {0.0: 0.206286836935167, 1.0: 0.793713163064833}}

In [ ]:
train_imgs, val_imgs, train_targets, val_targets = train_test_split(face_images, face_labels_enc,
                                                                    stratify=face_labels_enc,
                                                                    test_size=validation_split_size, random_state=100, shuffle=True)

train_imgs.shape, val_imgs.shape, train_targets.shape, val_targets.shape

((3257, 224, 224, 3), (815, 224, 224, 3), (3257, 3), (815, 3))

In [ ]:
# ensuring that the samples are stratified between train and test splits to validate the model right way
print(pd.DataFrame(train_targets).apply(pd.Series.value_counts, normalize=True))
print()
print(pd.DataFrame(val_targets).apply(pd.Series.value_counts, normalize=True))

            0         1         2
0.0  0.823764  0.969911  0.206325
1.0  0.176236  0.030089  0.793675

           0         1         2
0.0  0.82454  0.969325  0.206135
1.0  0.17546  0.030675  0.793865


In [ ]:
face_images, face_labels, face_labels_enc, face_locs, num_persons, targets, images, labels = None, None, None, None, None, None, None, None
del face_images, face_labels, face_labels_enc, face_locs, num_persons, targets, images, labels
# RAM usage after this = ~3.7GB (reduction of more than 3.5 GB)

making image generator

In [ ]:
train_image_generator = ImageDataGenerator(zoom_range=0.1, width_shift_range=0.1, height_shift_range=0.1,
                                           shear_range=0.15,fill_mode="nearest")

defining and training model

In [ ]:
vgg19_base = VGG19(include_top=False, pooling=None,
                   input_shape=(image_size, image_size, 3)) # with max pooling (None, 2048)


inner = vgg19_base.output


## only the followeing layers will be trained or weights updated will only be of below layers
inner = AveragePooling2D(pool_size=(7, 7))(inner)
inner = Flatten()(inner)
inner = Dense(units=256, activation='relu')(inner)
inner = Dropout(rate=0.25)(inner)
inner = Dense(units=3, activation='softmax')(inner)


model_1 = Model(inputs=vgg19_base.input, outputs=inner)


model_1.summary()

80142336/80134624 [==============================] - 2s 0us/step
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
______________________________________________________________

In [ ]:
model_1.compile(loss = 'categorical_crossentropy',                             # "multi log-loss"  as loss
                optimizer = Adam(lr=init_lr, decay=init_lr / num_epochs),      # "adam"            as optimiser
                metrics = [Recall(name='recall'), 'accuracy',
                           F1Score(average='macro', name='macro_f1', num_classes=3), # weighted_f1,
                           F1Score(average='weighted', name='weighted_f1', num_classes=3),
                           Precision(name='precision')])

In [ ]:
model_save_cb = ModelCheckpoint(filepath= save_vgg19+model_name+'-epoch{epoch:03d}-recall-{val_recall:.5f}-acc-{val_accuracy:.5f}.h5',
                                monitor='val_recall', mode='max', 
                                verbose=1, save_best_only=False, save_weights_only=True)
# storing the complete model to be able to resume training should something happens and also to load the model with best fbeta score on validation set for evaluation


# since recall is my primary metric of choice, i want the training to be stopped, when recall doesn't increase even after 15 epochs.
early_stop_cb = EarlyStopping(monitor='val_recall', min_delta=0, patience=20, verbose=1, mode='max')

In [ ]:
history_vgg19 = model_1.fit(train_image_generator.flow(x=train_imgs, y=train_targets, batch_size=batch_size, seed=100),
                            steps_per_epoch=len(train_imgs) // batch_size,
                            
                            validation_data = (val_imgs, val_targets),
                            validation_steps=len(val_imgs) // batch_size,
                            
                            epochs=num_epochs,
                            
                            class_weight={0:5, 1:13, 2:1}, # experimenting
                            
                            callbacks=[model_save_cb, early_stop_cb],
                            
                            verbose=2
                            )

Epoch 1/100


In [ ]:
# printing all the maximum scores
max(history_vgg19.history['val_recall']), max(history_vgg19.history['val_macro_f1']), max(history_vgg19.history['val_weighted_f1']), max(history_vgg19.history['val_accuracy'])

In [ ]:
train_stats = pd.DataFrame(history_vgg19.history)

# looking at the epochs that had best recall and macro-f1 scores for validaiton set
train_stats.sort_values(by=['val_recall'], inplace=False, ascending=False).head()

In [ ]:
train_stats.plot(y=['val_recall', 'recall'], kind="line")

In [ ]:
train_stats.plot(y=['val_macro_f1', 'macro_f1'], kind="line")

In [ ]:
train_stats.plot(y=['val_weighted_f1', 'weighted_f1'], kind="line")

In [ ]:
train_stats.plot(y=['val_accuracy', 'accuracy'], kind="line")

In [ ]:
train_stats.plot(y=['val_loss', 'loss'], kind="line")

In [ ]:
train_stats.plot(y=['val_precision', 'precision'], kind="line")

In [ ]:
very_good_epochs = []
for col in ['val_recall', 'val_accuracy','val_macro_f1', 'val_precision', 'val_weighted_f1']:
    epoch = train_stats.loc[:,col].argmax()
    very_good_epochs.append(epoch)
    print(train_stats.loc[epoch, ['val_recall', 'val_accuracy','val_macro_f1', 'val_weighted_f1']])
    print()

In [ ]:
# looking at all the rows with highest results for respective metric
good_results = train_stats.loc[set(very_good_epochs),
                               ['val_recall', 'val_accuracy','val_macro_f1', 'val_weighted_f1', 'val_precision']]

# since recall is my primary metric
good_results.sort_values(by=['val_recall', 'val_accuracy', 'val_macro_f1'], ascending=False, inplace=True)
good_results

In [ ]:
models_not_to_delete = []
for epoch in list(np.array(good_results.index)):
    good_vals = good_results.loc[epoch, ['val_recall', 'val_accuracy']].values
    best_model_loc = f'{save_vgg19}vgg_19-epoch{epoch+1:03d}-recall-{good_vals[0]:.5f}-acc-{good_vals[1]:.5f}.h5'
    print(best_model_loc)
    models_not_to_delete.append(best_model_loc)
    model_2 = None
    del model_2
    model_2 = None
    try:
        model_2 = Model(inputs=vgg19_base.input, outputs=inner)
        model_2.load_weights(filepath=best_model_loc)
        val_preds = model_2.predict(val_imgs, batch_size=32)
        val_preds = np.argmax(val_preds, axis=1)
        print(classification_report(y_true=val_targets.argmax(axis=1), y_pred=val_preds, target_names=['without mask', 'incorrectly worn', 'with mask']))
    except OSError:
        print('file not found')

In [ ]:
models_not_to_delete

In [ ]:
files.download('/content/VGG19/vgg_19-epoch047-recall-0.94847-acc-0.95215.h5')
files.download('/content/VGG19/vgg_19-epoch062-recall-0.94724-acc-0.94847.h5')
files.download('/content/VGG19/vgg_19-epoch065-recall-0.94356-acc-0.94724.h5')
files.download('/content/VGG19/vgg_19-epoch007-recall-0.90061-acc-0.93129.h5')

In [ ]:
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
img = image.load_img('/Hemakshi Janyani.png', target_size=(224, 224))

In [ ]:
img

In [ ]:
img_array = image.img_to_array(img)
img_batch = np.expand_dims(img_array, axis=0)

In [ ]:
img_batch

In [ ]:
img_preprocessed = preprocess_input(img_batch)

In [ ]:
img_preprocessed

In [ ]:
prediction = model_2.predict(img_preprocessed)

In [ ]:
prediction